In [34]:
import numpy as np
import matplotlib.pyplot as plt
import time
import tqdm

from disk_radon_test import *
from radon_transform import *

In [35]:
full_run = True

In [36]:
# Image parameters
image_height = 512
image_width = image_height
cmap = "gray"

In [37]:
xx, yy = np.mgrid[:image_height, :image_width]
center_x = image_width/2
center_y = image_height/2
SCALE = 45

In [38]:
def disk_func(x, y, radius):
    return x ** 2 + y ** 2 < radius ** 2

def p_function(x, y, a, delta_x, delta_y, radius):
    temp = np.array([disk_func(x-dx, y-dy, radius) for dx, dy in zip(delta_x, delta_y)])
    return (temp.T @ a).T

In [39]:
delta_x = np.array([1, 0.5, -0.5])
delta_y = 0.3 * np.array([0, 0.25, -0.5])
n = np.array([1, 0])
s = np.linspace(-2, 2, num=1000)
radius = 0.3
p_vals = p_function(xx-center_x, yy-center_y, np.ones(len(delta_x)), delta_x, delta_y, radius)
plt.imshow(p_vals[int(center_x-10):int(center_x+11), int(center_y-10):int(center_y+11)], cmap=cmap);

In [40]:
n = np.array([1, 0])
s = np.linspace(-2, 2, num=1000)

In [41]:
shifts = (128, 128)

In [42]:
disk = Disk(radius)
kwargs = {"support_disk_f": 2., "support_disk_t": 50.}
radon_gaussian = RadonTransform(disk, shifts, n.reshape(1,2), s, lattice_shifts = True, **kwargs)

In [43]:
%time
radon_p_vals = radon_gaussian.apply(np.ones(128 * 128))

In [44]:
plt.plot(s, radon_p_vals.flatten())
plt.title(label="R[p](n,s) with a fixed n = (1, 0)");

In [45]:
thetas = np.linspace(0, 2 * np.pi, num=100)
s = np.linspace(-1, 1, num=120)

In [46]:
radon_gaussian2 = RadonTransform(disk, shifts, np.array(list(zip(np.cos(thetas), np.sin(thetas)))), s, lattice_shifts=True, **kwargs)

In [47]:
%%time
results = radon_gaussian2.apply(np.ones(128 * 128))

In [48]:
plt.imshow(results.T, cmap=cmap)
plt.colorbar()
plt.ylabel("s")
plt.xlabel("n");

In [49]:
%%time
resultsF = radon_gaussian2.applyF(np.ones(128 * 128))

In [50]:
n_runs = 500
delta = 100
eps = 1e-6
N = disk.support(**kwargs)*disk.supportF(**kwargs)/2.
freqs = np.arange(-N, N + 1).reshape(-1, 1)
alphas = np.ones(128 * 128)

In [51]:
def time_run(k, radon_gaussian, time_step_1, time_step_2):
        start = time.time()
        resultsF = radon_gaussian.applyF(alphas)
        end = time.time()
        time_step_1[k] = end - start

        start = time.time()
        results = radon_gaussian.apply(alphas)
        end = time.time()
        time_step_2[k] = end - start - time_step_1[k]

In [52]:
if full_run:
    time_step_1 = np.zeros(delta)
    time_step_2 = np.zeros(delta)
    k = 0
    for run in tqdm.trange(n_runs - delta, n_runs):
        thetas = np.linspace(0, 2 * np.pi, num=run)
        s = np.linspace(-1, 1, num=10)
        radon_gaussian = RadonTransform(disk, shifts, np.array(list(zip(np.cos(thetas), np.sin(thetas)))), s, lattice_shifts = True, **kwargs)
        time_run(k, radon_gaussian, time_step_1, time_step_2)
        k += 1

In [53]:
def plot_benchmark(time_step_1, time_step_2, n_runs, delta, title):
    xs = np.arange(n_runs-delta, n_runs)
    fig, ax = plt.subplots( nrows=1, ncols=1 )
    plt.scatter(xs, time_step_1, label="step 1")
    plt.scatter(xs, time_step_2, label="step 2")
    m_s1, b_s1 = np.polyfit(xs, time_step_1, 1)
    m_s2, b_s2 = np.polyfit(xs, time_step_2, 1)
    plt.plot(xs, m_s1*xs+b_s1)
    plt.plot(xs, m_s2*xs+b_s2)
    plt.legend()
    plt.title(label=title)
    plt.show()
    fig.savefig("../../../outputs/" + title + ".png")

In [54]:
if full_run:
    plot_benchmark(time_step_1, time_step_2, n_runs, delta, "disk_griddeltas_increasing_num_n_eps1e-6")

In [55]:
if full_run:
    time_step_1 = np.zeros(delta)
    time_step_2 = np.zeros(delta)
    k = 0
    for run in tqdm.trange(n_runs - delta, n_runs):
        thetas = np.linspace(0, 2 * np.pi, num=30)
        s = np.linspace(-1, 1, num=run)
        radon_gaussian = RadonTransform(disk, shifts, np.array(list(zip(np.cos(thetas), np.sin(thetas)))), s, lattice_shifts=True, **kwargs)
        time_run(k, radon_gaussian, time_step_1, time_step_2)
        k += 1

In [56]:
if full_run:
    plot_benchmark(time_step_1, time_step_2, n_runs, delta, "disk_griddeltas_increasing_num_t_eps1e-6")